# 뉴스 제목-분류 크롤링

In [1]:
#정치 0
pol_url = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=100'
#경제 1
eco_url = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=101'
#사회 2
soc_url = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=102'
#생활/문화 3
lif_url = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=103'
#IT/과학 4
it_url = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105'

class_list = ['pol', 'eco', 'soc', 'lif', 'it']
url_list = [pol_url, eco_url, soc_url, lif_url, it_url]
target_class = ['cluster_text_headline nclicks(cls_pol.clsart)', 'cluster_text_headline nclicks(cls_eco.clsart)',
               'cluster_text_headline nclicks(cls_nav.clsart)', 'cluster_text_headline nclicks(cls_lif.clsart)',
               'cluster_text_headline nclicks(cls_sci.clsart)']

print('done')

done


In [2]:
#데이터셋 확보를 위해 주기적으로 돌려줄 것.

from bs4 import BeautifulSoup
import requests
import time
import re
import csv
import pandas as pd

for i, url in enumerate(url_list) :
    res = requests.get(url_list[i], headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
    })
    soup = BeautifulSoup(res.text, 'html.parser')
    result = soup.findAll('a', class_=target_class[i])
    
    for k, title in enumerate(result) : 
        result[k] = re.sub('<.+?>','',str(title)) #태그 제거
        result[k] = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', result[k]) #한글, 띄어쓰기 빼고 다 제거
    
    result_df = pd.DataFrame(result, columns=['title'])
    result_df['type'] = i
    
    exist_df = pd.read_csv('data/'+class_list[i]+'.csv')
    result_df = pd.concat([exist_df, result_df])
    
    result_df.drop(['Unnamed: 0'], axis=1, inplace=True) #합쳐지면서 이상한 열이 생기는 거 삭제

    result_df.drop_duplicates(subset=['title'], inplace=True) #중복 데이터 제거
    
    result_df.to_csv('data/'+class_list[i]+'.csv', mode='w', sep=',', na_rep='NaN')
    
    print(class_list[i], 'is saved!')
    time.sleep(3) #3초간 쉼으로써 크롤링 감지 및 차단 방지

pol is saved!
eco is saved!
soc is saved!
lif is saved!
it is saved!


In [3]:
#저장된 데이터 확인

for strs in class_list :
    df_data = pd.read_csv('data/'+strs+'.csv')
    print(df_data.shape)
    print(df_data)
    print(df_data['title'], df_data['type'])

(446, 3)
     Unnamed: 0                             title  type
0             0                  에서 쑥 들어간 홍남기 경질론     0
1             1     민주당 대통령 말씀 당과 일치재난지원금 선별보편 함께     0
2             2       선별이냐 보편이냐차 재난지원금 문 대통령의 속내는     0
3             3     차 재난지원금 선별보편 병행 관철  선별 차등지급해야     0
4             4         진성준 일체 신체접촉 없어민형사 책임 물을 것     0
..          ...                               ...   ...
441          35          조은희 는 정권 나팔수김어준 조선 많이 봤나     0
442          36    주호영 재보선 직전 재난지원금 국민 주권을 돈주고 사나     0
443          37           주호영 선거 앞두고 국민 주권 돈으로 사나     0
444          38           대통령 중기부가 손실보상제 방안 제안해달라     0
445          39  대통령 권칠승에게 손실보상제 합리적으로 만드는 것 큰 숙제     0

[446 rows x 3 columns]
0                      에서 쑥 들어간 홍남기 경질론
1         민주당 대통령 말씀 당과 일치재난지원금 선별보편 함께
2           선별이냐 보편이냐차 재난지원금 문 대통령의 속내는
3         차 재난지원금 선별보편 병행 관철  선별 차등지급해야
4             진성준 일체 신체접촉 없어민형사 책임 물을 것
                     ...               
441            조은희 는 정권 나팔수김어준 조선 많이 봤나
442      주호영 재보